In [19]:
from numpy import *
from scipy.sparse import *
import numpy as np

In [20]:
URM_all=load_npz("all_playlist_with_tracks_URM.npz")
trackIDs=load("fucking_ordered_tracks.npy")
playlistIDs=load("fucking_ordered_playlist.npy")
trackIDs.size,playlistIDs.size,URM_all

(100000, 45649, <45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 1040522 stored elements in Compressed Sparse Column format>)

In [21]:
#pBuilder=array([]).astype(int32)
#for i,p in enumerate(URM_all):
    #pBuilder=np.hstack((pBuilder,array([i]*p.size)))

In [22]:
#save("pBuilder.npy",pBuilder)
pBuilder=load("pBuilder.npy")
pBuilder.size

1040522

In [23]:
#tBuilder=array([]).astype(int32)
#for p in URM_all:
    #tBuilder=np.hstack((tBuilder,where(p.todense()!=0)[1]))

In [24]:
#save("tBuilder.npy",tBuilder)
tBuilder=load("tBuilder.npy")
tBuilder.size

1040522

In [25]:
rBuilder=np.ones(1040522).astype(int32)

In [26]:
class RandomRecommender(object):

    def fit(self, URM_train):
           
        self.numItems = URM_train.shape[1]
    
    
    def recommend(self, playlist_id, at=5):
        
        #genera 5 numeri da 0 a 99999
        recommended_index = np.random.choice(self.numItems, at)
        #prende le canzoni in quella posizione
        recommended_items = trackIDs[recommended_index]

        return recommended_items

In [27]:
class Top5Recommender(object):

    def recommend(self, playlist_id, at=5):
        
        recommended_items = array([1563309, 1363985, 3705881, 1595978, 3166665])

        return recommended_items

In [28]:
trackArtist=load_npz("fucking_ordered_track_artist.npz")
trackAlbum=load_npz("fucking_ordered_track_album.npz")
targetTrack=unique(genfromtxt("target_tracks.csv",dtype=int32,skip_header=1))
maskT=list(map(lambda x:x in targetTrack,trackIDs))

In [29]:
class ArtistAlbumRecommender(object):

    def fit(self, URM_train, artistWeight, albumWeight):
        
        simByArtist=trackArtist*trackArtist.T
        simByAlbum=trackAlbum*trackAlbum.T
        simByArtist.setdiag(0)
        simByAlbum.setdiag(0)
        S=artistWeight*simByArtist+albumWeight*simByAlbum
        RR=URM_train*S
        myMax=max([max(x.data) for x in RR if x.data.size!=0])
        self.RR=RR-URM_train*myMax
        
    def recommend(self, i, at=5):
        
        sims=self.RR[i]
        recommended_items = trackIDs[flip(argsort(sims.multiply(maskT).todense()),1).take(range(at))]
        
        return recommended_items

In [30]:
def precision(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

In [31]:
def recall(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

In [32]:
def MAP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

In [33]:
def evaluate_algorithm(URM_test, recommender_object, at=5):
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0
    
    num_eval = 0

    oldprogress=0
    for i,playlist_id in enumerate(playlistIDs):
        
        #va alla riga i-esima, che contiene la playlist attuale
        relevant_index = URM_test[i].indices
        #prende le track della playlist i-esima
        relevant_items = trackIDs[relevant_index]
        if len(relevant_items)>0:
            # HO MODIFICATO CON i
            #recommended_items = recommender_object.recommend(playlist_id, at=at)
            recommended_items = recommender_object.recommend(i, at=at)
            #print(str(playlist_id)+" "+str(recommended_items))
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_MAP += MAP(recommended_items, relevant_items)
        progress=i/45649*100
        if(progress-oldprogress>1):
            print("\r{:.2f}%".format(progress), end="")
            oldprogress=progress
        

    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval
    
    print("\r100%", end="")
    print(" Precision = {:.6f}, Recall = {:.6f}, MAP = {:.6f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP)) 


In [34]:
#ATTENZIONE, controllare che entrambe le matrici alla fine siano 45649x100000
# sennò si spacca tutto
train_test_split = 0.80
numInteractions = URM_all.nnz
train_mask = np.random.choice([True,False], numInteractions, p=[train_test_split, 1-train_test_split])

URM_train=(coo_matrix((rBuilder[train_mask],(pBuilder[train_mask],tBuilder[train_mask]))))
URM_train=URM_train.tocsr()

test_mask=logical_not(train_mask)
URM_test=(coo_matrix((rBuilder[test_mask],(pBuilder[test_mask],tBuilder[test_mask]))))
URM_test=URM_test.tocsr()
URM_train,URM_test

(<45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 832175 stored elements in Compressed Sparse Row format>,
 <45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 208347 stored elements in Compressed Sparse Row format>)

In [35]:
randomRecommender = RandomRecommender()
randomRecommender.fit(URM_train)
evaluate_algorithm(URM_test, randomRecommender)

100% Precision = 0.000062, Recall = 0.000059, MAP = 0.000034


In [36]:
top5recommender = Top5Recommender()
evaluate_algorithm(URM_test,top5recommender)

100% Precision = 0.002344, Recall = 0.001981, MAP = 0.001480


In [ ]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,0.5)
evaluate_algorithm(URM_test, artistAlbumRecommender)

In [ ]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,1)
evaluate_algorithm(URM_test, artistAlbumRecommender)

In [ ]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,2)
evaluate_algorithm(URM_test, artistAlbumRecommender)

In [ ]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,5)
evaluate_algorithm(URM_test, artistAlbumRecommender)

In [ ]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,10)
evaluate_algorithm(URM_test, artistAlbumRecommender)